<a href="https://colab.research.google.com/github/lalit2244/Local-Food-Wastage-Management-System/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Full Colab-ready script: SQL analysis + Excel export + charts embedded into Excel
# Run in Google Colab (copy-paste into a single cell).

# 0. Install dependencies
!pip install -q openpyxl pillow

# 1. Imports
from google.colab import files
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import os
from openpyxl import load_workbook
from openpyxl.drawing.image import Image as OpenpyxlImage
import datetime

# 2. Upload CSV files
print("Upload: providers_data.csv, receivers_data.csv, food_listings_data.csv, claims_data.csv")
uploaded = files.upload()  # use the upload dialog

# 3. Read CSVs (assumes exact filenames)
providers = pd.read_csv('providers_data.csv')
receivers = pd.read_csv('receivers_data.csv')
food_listings = pd.read_csv('food_listings_data.csv')
claims = pd.read_csv('claims_data.csv')

# 4. Create SQLite DB and store tables
conn = sqlite3.connect('foodwastage.db')
providers.to_sql('providers', conn, if_exists='replace', index=False)
receivers.to_sql('receivers', conn, if_exists='replace', index=False)
food_listings.to_sql('food_listings', conn, if_exists='replace', index=False)
claims.to_sql('claims', conn, if_exists='replace', index=False)

# 5. Define queries (15)
queries = {
    "Q1_Providers_Receivers_by_City": """
        SELECT City,
               COUNT(DISTINCT Provider_ID) AS providers_count,
               (SELECT COUNT(DISTINCT Receiver_ID) FROM receivers r WHERE r.City = p.City) AS receivers_count
        FROM providers p
        GROUP BY City
    """,
    "Q2_Top_Contributing_Provider_Type": """
        SELECT Provider_Type, SUM(Quantity) AS total_quantity
        FROM food_listings
        GROUP BY Provider_Type
        ORDER BY total_quantity DESC
        LIMIT 1
    """,
    "Q3_Contact_Info_Providers": """
        SELECT Name, City, Contact
        FROM providers
        ORDER BY City
    """,
    "Q4_Receivers_with_Most_Claims": """
        SELECT r.Name, COUNT(c.Claim_ID) AS total_claims
        FROM claims c
        JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
        GROUP BY r.Name
        ORDER BY total_claims DESC
    """,
    "Q5_Total_Food_Available": """
        SELECT SUM(Quantity) AS total_available_quantity
        FROM food_listings
    """,
    "Q6_City_with_Most_Listings": """
        SELECT Location, COUNT(*) AS listings_count
        FROM food_listings
        GROUP BY Location
        ORDER BY listings_count DESC
        LIMIT 1
    """,
    "Q7_Most_Common_Food_Types": """
        SELECT Food_Type, COUNT(*) AS count_type
        FROM food_listings
        GROUP BY Food_Type
        ORDER BY count_type DESC
    """,
    "Q8_Claims_per_Food_Item": """
        SELECT f.Food_Name, COUNT(c.Claim_ID) AS total_claims
        FROM claims c
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        GROUP BY f.Food_Name
    """,
    "Q9_Top_Provider_Successful_Claims": """
        SELECT p.Name, COUNT(c.Claim_ID) AS successful_claims
        FROM claims c
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        JOIN providers p ON f.Provider_ID = p.Provider_ID
        WHERE c.Status = 'Completed'
        GROUP BY p.Name
        ORDER BY successful_claims DESC
        LIMIT 1
    """,
    "Q10_Claim_Status_Distribution": """
        SELECT Status, COUNT(*) AS status_count,
               ROUND( (COUNT(*) * 100.0) / (SELECT COUNT(*) FROM claims), 2 ) AS percentage
        FROM claims
        GROUP BY Status
    """,
    "Q11_Avg_Quantity_per_Receiver": """
        SELECT r.Name, ROUND(AVG(f.Quantity), 2) AS avg_quantity
        FROM claims c
        JOIN receivers r ON c.Receiver_ID = r.Receiver_ID
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        GROUP BY r.Name
    """,
    "Q12_Most_Claimed_Meal_Type": """
        SELECT Meal_Type, COUNT(*) AS claim_count
        FROM claims c
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        GROUP BY Meal_Type
        ORDER BY claim_count DESC
        LIMIT 5
    """,  # return top-5 for plotting
    "Q13_Total_Donated_by_Provider": """
        SELECT p.Name, SUM(f.Quantity) AS total_donated
        FROM food_listings f
        JOIN providers p ON f.Provider_ID = p.Provider_ID
        GROUP BY p.Name
        ORDER BY total_donated DESC
    """,
    "Q14_Top5_Food_Items_by_Quantity": """
        SELECT Food_Name, SUM(Quantity) AS total_qty
        FROM food_listings
        GROUP BY Food_Name
        ORDER BY total_qty DESC
        LIMIT 5
    """,
    "Q15_Cities_with_Most_Completed_Claims": """
        SELECT f.Location, COUNT(c.Claim_ID) AS completed_claims
        FROM claims c
        JOIN food_listings f ON c.Food_ID = f.Food_ID
        WHERE c.Status = 'Completed'
        GROUP BY f.Location
        ORDER BY completed_claims DESC
    """
}

# 6. Execute queries and collect results
query_results = {}
for name, sql in queries.items():
    df = pd.read_sql(sql, conn)
    query_results[name] = df
    print(f"\n--- {name} ---")
    display(df.head(10))

# 7. Prepare folder for charts
img_folder = "charts"
os.makedirs(img_folder, exist_ok=True)

# 8. Create charts and save as PNGs.
#    Use safe checks in case a query returned empty DataFrame.

# Chart 1: Providers & Receivers by City (from Q1)
df1 = query_results.get("Q1_Providers_Receivers_by_City", pd.DataFrame())
if not df1.empty:
    ax = df1.set_index('City')[['providers_count','receivers_count']].plot(kind='bar', figsize=(9,5))
    plt.title("Providers & Receivers by City")
    plt.ylabel("Count")
    plt.tight_layout()
    path = os.path.join(img_folder, "Q1_providers_receivers_by_city.png")
    plt.savefig(path)
    plt.close()
else:
    path = None

# Chart 2: Claim Status Distribution (from Q10)
df10 = query_results.get("Q10_Claim_Status_Distribution", pd.DataFrame())
if not df10.empty:
    plt.figure(figsize=(6,6))
    plt.pie(df10['status_count'], labels=df10['Status'], autopct='%1.1f%%', startangle=90)
    plt.title("Claim Status Distribution")
    plt.tight_layout()
    path10 = os.path.join(img_folder, "Q10_claim_status_distribution.png")
    plt.savefig(path10)
    plt.close()
else:
    path10 = None

# Chart 3: Top 5 Food Types (from Q7)
df7 = query_results.get("Q7_Most_Common_Food_Types", pd.DataFrame())
if not df7.empty:
    df7_top5 = df7.head(5)
    ax = df7_top5.plot(x='Food_Type', y='count_type', kind='bar', figsize=(8,5))
    plt.title("Top 5 Food Types")
    plt.ylabel("Count")
    plt.tight_layout()
    path7 = os.path.join(img_folder, "Q7_top5_food_types.png")
    plt.savefig(path7)
    plt.close()
else:
    path7 = None

# Chart 4: Top 5 Providers by Total Donations (from Q13)
df13 = query_results.get("Q13_Total_Donated_by_Provider", pd.DataFrame())
if not df13.empty:
    df13_top5 = df13.head(5).iloc[::-1]  # reverse for horizontal bar order
    ax = df13_top5.plot(x='Name', y='total_donated', kind='barh', figsize=(8,5))
    plt.title("Top 5 Providers by Total Donations")
    plt.xlabel("Total Quantity Donated")
    plt.tight_layout()
    path13 = os.path.join(img_folder, "Q13_top5_providers.png")
    plt.savefig(path13)
    plt.close()
else:
    path13 = None

# Chart 5: Meal Type Claims (from Q12)
df12 = query_results.get("Q12_Most_Claimed_Meal_Type", pd.DataFrame())
if not df12.empty:
    plt.figure(figsize=(6,6))
    plt.pie(df12['claim_count'], labels=df12['Meal_Type'], autopct='%1.1f%%', startangle=90)
    plt.title("Top Meal Type Claims")
    plt.tight_layout()
    path12 = os.path.join(img_folder, "Q12_meal_type_claims.png")
    plt.savefig(path12)
    plt.close()
else:
    path12 = None


# Map sheet name -> image path (if exists)
sheet_image_map = {
    "Q1_Providers_Receivers_by_City": path,
    "Q10_Claim_Status_Distribution": path10,
    "Q7_Most_Common_Food_Types": path7,
    "Q13_Total_Donated_by_Provider": path13,
    "Q12_Most_Claimed_Meal_Type": path12
}

# 9. Save all query results to an Excel file (one sheet per query)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
excel_filename = f"foodwastage_analysis_with_charts_{timestamp}.xlsx"

with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    for sheet_name, df in query_results.items():
        safe_sheet = sheet_name[:31]  # Excel limit
        # If empty dataframe, write an indicator row
        if df.empty:
            pd.DataFrame({"Info": ["No data for this query"]}).to_excel(writer, sheet_name=safe_sheet, index=False)
        else:
            df.to_excel(writer, sheet_name=safe_sheet, index=False)
print(f"\nSaved query results to Excel: {excel_filename}")

# 10. Embed saved chart images into the corresponding sheets using openpyxl
wb = load_workbook(excel_filename)
for sheet_name, img_path in sheet_image_map.items():
    if img_path and os.path.exists(img_path):
        safe_sheet = sheet_name[:31]
        if safe_sheet in wb.sheetnames:
            ws = wb[safe_sheet]
            # Insert image at cell H2 (column 8). If H2 is too far, Excel will handle layout.
            img = OpenpyxlImage(img_path)
            img.anchor = 'H2'
            ws.add_image(img)
# Save workbook with images
wb.save(excel_filename)
print(f"Embedded charts (where available) into: {excel_filename}")

# 11. Close DB connection
conn.close()

# 12. Provide the file for download
files.download(excel_filename)

# 13. Optional: show where images are saved
print("\nChart images saved in folder:", img_folder)
print("Done. Download your Excel file from the browser download prompt.")

Upload: providers_data.csv, receivers_data.csv, food_listings_data.csv, claims_data.csv


Saving claims_data.csv to claims_data (1).csv
Saving food_listings_data.csv to food_listings_data (1).csv
Saving receivers_data.csv to receivers_data (1).csv
Saving providers_data.csv to providers_data (3).csv

--- Q1_Providers_Receivers_by_City ---


,City,providers_count,receivers_count
0,Adambury,1,0
1,Adamsview,1,0
2,Adamsville,1,0
3,Aguirreville,1,0
4,Alexanderchester,1,0
5,Alexanderstad,1,0
6,Allenborough,1,1
7,Allenton,1,0
8,Amandaborough,1,0
9,Amandashire,1,0



--- Q2_Top_Contributing_Provider_Type ---


,Provider_Type,total_quantity
0,Restaurant,6923


...

,Location,completed_claims
0,South Kathryn,5
1,Zimmermanville,4
2,New Carol,4
3,East Heatherport,4
4,Devinmouth,4
5,Coleburgh,4
6,Shortfurt,3
7,Phillipsfort,3
8,Perezport,3
9,Patrickfort,3



Saved query results to Excel: foodwastage_analysis_with_charts_20250814_052028.xlsx
Embedded charts (where available) into: foodwastage_analysis_with_charts_20250814_052028.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Chart images saved in folder: charts
Done. Download your Excel file from the browser download prompt.


In [ ]:
    !pip install -q streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

def get_connection():
    return sqlite3.connect('foodwastage.db')

def home():
    st.title("🍽 Local Food Wastage Management System")
    conn = get_connection()
    df = pd.read_sql("SELECT * FROM food_listings", conn)
    st.dataframe(df)
    conn.close()

menu = ["Home"]
choice = st.sidebar.radio("Navigate", menu)
if choice == "Home":
    home()


Overwriting app.py


In [ ]:
import os
import threading

# Function to run Streamlit in background
def run_streamlit():
    os.system("streamlit run app.py")

threading.Thread(target=run_streamlit).start()


In [ ]:
!lt --port 8501


your url is: https://yellow-ducks-read.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:30555 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0


In [ ]:
!pip install streamlit pyngrok --quiet


In [ ]:
!ngrok authtoken 31GWQWU0BraCpBNrJRRAumqKfbS_3oJ3JnvVLaFdtBSyRZat1


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
%%writefile app.py
import streamlit as st
st.title("🍽 Local Food Wastage Management System")
st.write("This is running in Colab via Streamlit & ngrok.")

# Example input/output
name = st.text_input("Your Name")
if st.button("Greet Me"):
    st.success(f"Hello, {name}!")

# Replace this with your full Streamlit app code


Overwriting app.py


In [ ]:
!pip install pyngrok==5.2.1 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 31.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install --upgrade streamlit --quiet


In [ ]:
!ngrok authtoken 31GWQWU0BraCpBNrJRRAumqKfbS_3oJ3JnvVLaFdtBSyRZat1


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from pyngrok import ngrok
import threading
import os

# Run Streamlit in background
def run_streamlit():
    os.system("streamlit run app.py")

threading.Thread(target=run_streamlit).start()

# Create public URL
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://1d58b85c504b.ngrok-free.app" -> "http://localhost:8501">

In [ ]:
# Stop previous thread if needed and rerun
import threading, os
threading.Thread(target=lambda: os.system("streamlit run app.py")).start()


In [ ]:
from pyngrok import ngrok
import threading
import os

# Run Streamlit in background
def run_streamlit():
    os.system("streamlit run app.py")

threading.Thread(target=run_streamlit).start()

# Create public URL
public_url = ngrok.connect(8501)
public_url

<NgrokTunnel: "https://9cae7dac5965.ngrok-free.app" -> "http://localhost:8501">

In [ ]:
import threading, os
threading.Thread(target=lambda: os.system("streamlit run app.py")).start()


In [ ]:
!pip install streamlit pandas matplotlib pyngrok --quiet


In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# --------------------------
# DB Connection Function
# --------------------------
def get_connection():
    return sqlite3.connect('foodwastage.db')

# --------------------------
# Home Page
# --------------------------
def home():
    st.title("🍽 Local Food Wastage Management System")
    st.write("Connecting surplus food providers to those in need, reducing waste.")

    conn = get_connection()
    cities = ["All"] + list(pd.read_sql("SELECT DISTINCT City FROM providers", conn)['City'])
    food_types = ["All"] + list(pd.read_sql("SELECT DISTINCT Food_Type FROM food_listings", conn)['Food_Type'])
    conn.close()

    city = st.sidebar.selectbox("Filter by City", cities)
    ftype = st.sidebar.selectbox("Filter by Food Type", food_types)

    conn = get_connection()
    query = "SELECT * FROM food_listings"
    conditions = []
    if city != "All":
        conditions.append(f"Location = '{city}'")
    if ftype != "All":
        conditions.append(f"Food_Type = '{ftype}'")
    if conditions:
        query += " WHERE " + " AND ".join(conditions)
    df = pd.read_sql(query, conn)
    conn.close()

    st.write("### Available Food Listings")
    st.dataframe(df)

# --------------------------
# CRUD Operations
# --------------------------
def crud():
    st.title("🛠 Manage Food Listings")
    conn = get_connection()

    # Add
    st.subheader("➕ Add New Food Listing")
    with st.form("add_food"):
        food_name = st.text_input("Food Name")
        qty = st.number_input("Quantity", min_value=1)
        expiry = st.date_input("Expiry Date")
        provider_id = st.number_input("Provider ID", min_value=1)
        provider_type = st.text_input("Provider Type")
        location = st.text_input("City")
        ftype = st.text_input("Food Type")
        mtype = st.text_input("Meal Type")
        submitted = st.form_submit_button("Add")
        if submitted:
            conn.execute(
                "INSERT INTO food_listings (Food_Name, Quantity, Expiry_Date, Provider_ID, Provider_Type, Location, Food_Type, Meal_Type) VALUES (?, ?, ?, ?, ?, ?, ?, ?)",
                (food_name, qty, expiry, provider_id, provider_type, location, ftype, mtype)
            )
            conn.commit()
            st.success("✅ Food listing added!")

    # Read
    st.subheader("📋 Current Food Listings")
    df = pd.read_sql("SELECT * FROM food_listings", conn)
    st.dataframe(df)

    # Update
    st.subheader("✏️ Update Food Listing")
    ids = df['Food_ID'].tolist()
    if ids:
        selected_id = st.selectbox("Select Food ID to Update", ids)
        new_qty = st.number_input("New Quantity", min_value=1)
        if st.button("Update Quantity"):
            conn.execute("UPDATE food_listings SET Quantity = ? WHERE Food_ID = ?", (new_qty, selected_id))
            conn.commit()
            st.success("✅ Updated successfully!")

    # Delete
    st.subheader("🗑 Delete Food Listing")
    if ids:
        del_id = st.selectbox("Select Food ID to Delete", ids, key="delete")
        if st.button("Delete Listing"):
            conn.execute("DELETE FROM food_listings WHERE Food_ID = ?", (del_id,))
            conn.commit()
            st.warning("⚠️ Listing deleted!")

    conn.close()

# --------------------------
# SQL Queries Tab
# --------------------------
def queries_tab():
    st.title("📊 SQL Queries")
    conn = get_connection()

    queries = {
        "Providers by City": "SELECT City, COUNT(*) AS Total_Providers FROM providers GROUP BY City",
        "Total Food Quantity": "SELECT SUM(Quantity) AS Total_Quantity FROM food_listings",
        "Top Food Types": "SELECT Food_Type, COUNT(*) AS Count FROM food_listings GROUP BY Food_Type ORDER BY Count DESC"
    }

    for title, sql in queries.items():
        st.subheader(title)
        df = pd.read_sql(sql, conn)
        st.dataframe(df)

    conn.close()

# --------------------------
# Charts Tab
# --------------------------
def charts_tab():
    st.title("📈 Charts")
    conn = get_connection()

    # Claim Status Pie Chart
    df_status = pd.read_sql("SELECT Status, COUNT(*) AS Count FROM claims GROUP BY Status", conn)
    if not df_status.empty:
        fig1, ax1 = plt.subplots()
        ax1.pie(df_status['Count'], labels=df_status['Status'], autopct='%1.1f%%', startangle=90)
        ax1.set_title("Claim Status Distribution")
        st.pyplot(fig1)

    # Top Providers Bar Chart
    df_providers = pd.read_sql("""
        SELECT p.Name, SUM(f.Quantity) AS Total_Quantity
        FROM food_listings f
        JOIN providers p ON f.Provider_ID = p.Provider_ID
        GROUP BY p.Name
        ORDER BY Total_Quantity DESC
        LIMIT 5
    """, conn)
    if not df_providers.empty:
        fig2, ax2 = plt.subplots()
        ax2.barh(df_providers['Name'], df_providers['Total_Quantity'], color='green')
        ax2.set_xlabel("Total Quantity")
        ax2.set_title("Top 5 Providers by Donations")
        st.pyplot(fig2)

    conn.close()

# --------------------------
# App Navigation
# --------------------------
menu = ["Home", "Manage Data", "SQL Queries", "Charts"]
choice = st.sidebar.radio("Navigate", menu)

if choice == "Home":
    home()
elif choice == "Manage Data":
    crud()
elif choice == "SQL Queries":
    queries_tab()
elif choice == "Charts":
    charts_tab()


Overwriting app.py


In [ ]:
from pyngrok import ngrok
import threading, os

# Kill old tunnels
ngrok.kill()

# Start Streamlit
def run_streamlit():
    os.system("streamlit run app.py")

threading.Thread(target=run_streamlit).start()

# Create public URL
public_url = ngrok.connect(8501)
public_url


<NgrokTunnel: "https://7f70caaba655.ngrok-free.app" -> "http://localhost:8501">